# Getting Started with Modern Management of Microsoft 365

 > You will review some introductory concepts in working with Microsoft 365 services through the Microsoft Graph API and then combine these services for more complex use cases.

# Table of Contents
 - [Starting with basics](#starting-with-basics)
   - [Install Modules](#install-modules)
   - [Manage Permissions](#manage-permissions)
   - [Get Email](#get-email)
   - [Send Teams Message](#send-teams-message)
 - [Combining these for complex uses](#combining-these-for-complex-uses)
   - [Run a Report](#run-a-report)
   - [Summarize Report](#summarize-report)
   - [Send Report Summary to Teams](#send-report-summary-to-teams)

In [ ]:
flowchart TB
    subgraph one [Starting with basics]
        direction LR
        s1(Install Modules) ==> s2(Manage Permissions)
        s2 ==> s3(Get Email)
        s3 ==> s4(Send Teams Message)
    end
    subgraph two [Combining these for complex uses]
        direction LR
        s5(Run a Report) ==> s6(Summarize Report)
        s6 ==> s7(Send Report to Teams)
    end
    one ==> two

# Starting with basics

## Install Modules

You will use PowerShell to work with the [Microsoft Graph API](https://learn.microsoft.com/en-us/graph/overview) and to do this you will require the use of a few modules.

Begin by:
 - Identifying the required modules using the [array sub-expression operator](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_arrays#the-array-sub-expression-operator)

In [ ]:
$modules = @(
    "Microsoft.Graph.Authentication",
    "Microsoft.Graph.Mail",
    "Microsoft.Graph.Teams",
    "Microsoft.Graph.Reports"
)

 - Identifying currently available Graph modules & trust the PSGallery

> ---
> ✒️ **_NOTE:_** Below you will use the `?` alias for Where-Object and the `$_` alias for [`$PSItem`](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_psitem)
>
> ---

In [ ]:
$available = Get-Module -ListAvailable|?{$_.Name -like "Microsoft.Graph.*"}
Set-PSRepository -Name 'PSGallery' -InstallationPolicy Trusted

 - Then if a necessary module is not available you will install it

In [ ]:
foreach($module in $modules)
{
    if($module -notin $available.Name)
    {
        Install-Module $module
    }
}

## Manage Permissions

Now that you have the necessary modules, you can authenticate and be authorized for the different commands you will run.

 - Create an array of the commands you will need to run

In [ ]:
$commands = @(
    "Get-MgUserMessage",
    "New-MgChatMessage",
    "Get-MgReportEmailActivityUserDetail"
)

 - For each of your commands use the `Find-MgGraphCommand` to identify the permissions necessary, add them to the `$permissions` array, and keep the list unique

In [ ]:
$permissions = @()
foreach ($command in $commands)
{
    $p = (Find-MgGraphCommand -Command $command).Permissions.Name
    $permissions += $p | Select-Object -Unique
    $permissions = $permissions | Select-Object -Unique | Sort-Object
}

 - Connect to the Microsoft Graph and include the permissions you need authorization for

 > ---
> ⚠️ **_Warning:_** You will login to an interactive webpage for the next codeblock
>
> ---

In [ ]:
Connect-MgGraph -Scopes $permissions -ContextScope Process

## Get Email

Now that you have authenticated and been authorized to work with the Microsoft Graph, you will read an email from an Exchange Online Mailbox.

 - Start by getting the user identity which you authenticated to the graph as

In [ ]:
$userId = (Get-MgContext).Account

 - Get the most recent email in your mailbox and display the from address, subject, and body preview

In [ ]:
$messages = Get-MgUserMessage -UserId "$userId" -Top 1
$messages | Select-Object @{n='From';e={$_.From.EmailAddress.Address}},Subject,BodyPreview

## Send Teams Message

Lets work with Microsoft Teams as your next Office 365 service through the Microsoft Graph. You will take that email and share it as a Teams Message.

 - Define the channel you want the message to go to as `$chatId` and the message contents as `$message`
   - You will want to change the chatId to the channel that is correct for your tenant, it is easiest to find this by using the Teams web client
 
> ---
> ✒️ **_NOTE:_** The `$chatId` variable defined below is used in following sections.
>
> ---

In [ ]:
$chatId = "19:669708bdb81e46ccbd58e05ec1bdcb44@thread.tacv2"
$message = @{
    ContentType = "html"
    Content = @"
        <h2>New email message</h2>
        <p><b>From:</b> $(($messages).From.EmailAddress.Address)</p>
        <p><b>To:</b> $(($messages).ToRecipients.EmailAddress.Address)</p>
        <p><b>Subject:</b> $(($messages).Subject)</p>
        <p><b>Body Preview:</b> $(($messages).BodyPreview)</p>
"@
}

 - Send your message to Teams

In [ ]:
New-MgChatMessage -ChatId "$chatId"`
    -Body $message

# Combining these for complex uses

## Run a Report

Within the Microsoft 365 Admin Center there are many graphs and [activity reports](https://learn.microsoft.com/en-us/graph/api/reportroot-getemailactivityuserdetail) available. You can dynamically work with these reports and add them to your normal operations using the Microsoft Graph API.

 - Begin by getting the email activity details by user for the past 30 days to the `temp.csv` in the current directory

In [ ]:
Get-MgReportEmailActivityUserDetail -Period "D30" -OutFile ./temp.csv

## Summarize Report

Now that you have a copy of the report downloaded you can summarize the contents.

 - Import the `temp.csv` report to a PowerShell object and then generate some measurements of the objects

In [ ]:
$report = Import-Csv ./temp.csv
$summary = $report|Measure-Object 'Send Count',`
                        'Receive Count',`
                        'Read Count',`
                        'Meeting Created Count',`
                        'Meeting Interacted Count' -Average -Sum -Maximum -Minimum

 - You can check the summary was properly calculated and display it in an HTML table in preparation for sending it in Teams

In [ ]:
$summary|select Property,Count,Sum,Maximum,Minimum,Average|ConvertTo-Html -Fragment

## Send Report Summary to Teams

With your summarized email usage, now you can share this with the team.

 - Include your summary in the message body and send your message

In [ ]:
$message = @{
    ContentType = "html"
    Content = @"
        <h2>Monthly Email Utilization Report</h2>
        <p>
        $($summary|Select-Object Property,Count,Sum,Maximum,Minimum,Average|ConvertTo-Html -Fragment)
        </p>
"@
}
New-MgChatMessage -ChatId "$chatId"`
    -Body $message

 - Last but not least clean up the temp file you created

In [ ]:
Remove-Item ./temp.csv

This is just the beginning of what you can do with the Microsoft Graph and other Microsoft 365 management techniques using PowerShell.